# $N$-QPU VQE - Vertical Speedup

This notebook implements routines for running a VQE ansatz on several Quantum Computers of smaller size, which are connected both quantumly and classically, thus achieving a vertical speedup, using Interlin-q. Here, we first use a greedy algorithm to see how many qubits from each QPU do we need, and then use this information to build our parallel schedule accordingly. The controller host then takes care of sending the schedule to the different QPU so that each one knows their respective tasks. Since the networking overhead in this approach is relatively big, we will manually retrieve the statevector from the QPU network and calculate the expecation value ourselves.

In this notebook, we will be applying VQE for the Hamiltonian of the Hydrogen molecule, which requires a maximum of 4 qubits for its ansatz, which is displayed below:

<img src="https://pennylane.ai/qml/_images/sketch_circuit.png" alt="H2 ansatz" style="width:700px;"/>

## Step 1: Import libraries.

First we import all the necessary libraries. Interlin-q is built using the Python framework [QuNetSim](https://arxiv.org/abs/2003.06397) which is a software framework for simulating quantum networks up to the network layer. We also need PennyLane's chemistry library for decomposing the Hamiltonian. In addition, we import the scheduling algorithm which is based on Algorithm 1 from this [paper](https://arxiv.org/pdf/2101.02504.pdf).

In [1]:
%load_ext autoreload
%autoreload 2

# Basic Libraries
import sys
import numpy as np

sys.path.append("../../")

# QuNetSim Components
from qunetsim.components import Network
from qunetsim.objects import Logger
from qunetsim.backends.eqsn_backend import EQSNBackend

# Interlin-q Components
from interlinq import (ControllerHost, Constants, Clock,
Circuit, Layer, ComputingHost, Operation)

# Extra needed components
from hamiltonian_decomposition import decompose
from general_scheduler import HardwareConfig, GreedySchedule

Logger.DISABLED = False

In [2]:
import pennylane as qml
qml.version() # should be 0.15.1. Higher versions are not integrated with the notebook yet

'0.15.1'

## Step 2: Decompose the Hamiltonian.

In [3]:
# These parameters are mentioned in PennyLane's VQE tutorial (https://pennylane.ai/qml/demos/tutorial_vqe.html) 
# and are used as is for benchmarking
geometry = 'h2.xyz'
charge = 0
multiplicity = 1
basis_set = 'sto-3g'
name = 'h2'

In [4]:
# The decompose function runs PennyLane's decomposers and strips out the observables from their PennyLane objects
# to be used easily for our purposes
coefficients, observables, qubit_num = decompose(name, geometry, charge, multiplicity, basis_set)
terms = list(zip(coefficients, observables))

terms

[(-0.04207897647782276, [('Identity', 0)]),
 (0.17771287465139946, [('PauliZ', 0)]),
 (0.1777128746513994, [('PauliZ', 1)]),
 (-0.24274280513140462, [('PauliZ', 2)]),
 (-0.24274280513140462, [('PauliZ', 3)]),
 (0.17059738328801052, [('PauliZ', 0), ('PauliZ', 1)]),
 (0.04475014401535161,
  [('PauliY', 0), ('PauliX', 1), ('PauliX', 2), ('PauliY', 3)]),
 (-0.04475014401535161,
  [('PauliY', 0), ('PauliY', 1), ('PauliX', 2), ('PauliX', 3)]),
 (-0.04475014401535161,
  [('PauliX', 0), ('PauliX', 1), ('PauliY', 2), ('PauliY', 3)]),
 (0.04475014401535161,
  [('PauliX', 0), ('PauliY', 1), ('PauliY', 2), ('PauliX', 3)]),
 (0.12293305056183798, [('PauliZ', 0), ('PauliZ', 2)]),
 (0.1676831945771896, [('PauliZ', 0), ('PauliZ', 3)]),
 (0.1676831945771896, [('PauliZ', 1), ('PauliZ', 2)]),
 (0.12293305056183798, [('PauliZ', 1), ('PauliZ', 3)]),
 (0.17627640804319591, [('PauliZ', 2), ('PauliZ', 3)])]

## Step 3: Schedule our Observables on our QPUs

We are going to assume a very simple quantum network consisting of merely 3 QPUs, where a pair has two qubits and the third QPU has only one qubit.

In [5]:
# First, determine the size of the ansatz
max = 0
for term in terms:
    _, obs = term
    
    for ob in obs:
        _, idx = ob
        
        max = idx if idx > max else max
max += 1

max

4

In [6]:
number_of_observables = len(terms)
hardware_configuration = [2, 2, 1]

In [7]:
config = HardwareConfig(hardware_configuration)
sched = GreedySchedule(number_of_observables, config, max, True)
sched.make_schedule()

sched.print_schedule()

### Schedule for parallelization ###
# Round 1 #
[(0, [2, 2, 0])]
# Round 2 #
[(0, [2, 2, 0])]
# Round 3 #
[(0, [2, 2, 0])]
# Round 4 #
[(0, [2, 2, 0])]
# Round 5 #
[(0, [2, 2, 0])]
# Round 6 #
[(0, [2, 2, 0])]
# Round 7 #
[(0, [2, 2, 0])]
# Round 8 #
[(0, [2, 2, 0])]
# Round 9 #
[(0, [2, 2, 0])]
# Round 10 #
[(0, [2, 2, 0])]
# Round 11 #
[(0, [2, 2, 0])]
# Round 12 #
[(0, [2, 2, 0])]
# Round 13 #
[(0, [2, 2, 0])]
# Round 14 #
[(0, [2, 2, 0])]
# Round 15 #
[(0, [2, 2, 0])]



As expected, we only need two qubits each from the bigger QPUs for our ansatz. And so we will build our circuits accordingly.

## Step 4: Prepare Circuit for Given Parameters

The circuit can be prepared using two different ways: either as one circuit, or several circuits run sequentially. The former approach is simpler and generally better for the optimisation function. The latter is better for debugging and for dynamic components of a quantum circuit (i.e. circuits that have a lot of changing operations). For the rest of this notebook, we will use the former approach, since the networking overhead can be computationally expensive in a threaded environment.

### Main Blocks

In [8]:
# Arbitrary single qubit rotation, as implemented by PennyLane here: 
# https://pennylane.readthedocs.io/en/stable/code/api/pennylane.Rot.html#pennylane.Rot
def rotational_gate(params):
    phi, theta, omega = params
    cos = np.cos(theta / 2)
    sin = np.sin(theta / 2)
    
    res = np.array([[np.exp(-1j * (phi + omega) / 2) * cos, -np.exp(1j * (phi - omega) / 2) * sin], 
                     [np.exp(-1j * (phi - omega) / 2) * sin, np.exp(1j * (phi + omega) / 2) * cos]])
    
    return res

In [9]:
# These operations are responsible for preparing the initial state of the qubits, as shown in the figure above.
# We assume the qubits are distributed consecutively 
def initialisation_operations(q_map):
    computing_host_ids = list(q_map.keys())
    
    ops = []
    for host_id in computing_host_ids:
        # We first initialize all the qubits in the backend
        op = Operation(
            name=Constants.PREPARE_QUBITS,
            qids=q_map[host_id],
            computing_host_ids=[host_id])
        ops.append(op)
    
    ### Workaround for a bug in EQSN
    # The first two in the first host
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map['QPU_0'][0], q_map['QPU_0'][1]],
        gate=Operation.CNOT,
        computing_host_ids=['QPU_0'])
    ops.append(op)
    
    # Between the two computers
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map['QPU_0'][1], q_map['QPU_1'][0]],
        gate=Operation.CNOT,
        computing_host_ids=computing_host_ids)
    ops.append(op)

    # The two in the second computer
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map['QPU_1'][0], q_map['QPU_1'][1]],
        gate=Operation.CNOT,
        computing_host_ids=['QPU_1'])
    ops.append(op)
    ################################

    # Prepare the qubits on the computing host
    op = Operation(
        name=Constants.SINGLE,
        qids=[q_map['QPU_0'][0]],
        gate=Operation.X,
        computing_host_ids=['QPU_0'])
    ops.append(op)
    
    op = Operation(
        name=Constants.SINGLE,
        qids=[q_map['QPU_0'][1]],
        gate=Operation.X,
        computing_host_ids=['QPU_0'])
    ops.append(op)
    
    # Needed to fix bug in EQSN
    op = Operation(
        name=Constants.SINGLE,
        qids=[q_map['QPU_1'][0]],
        gate=Operation.X,
        computing_host_ids=['QPU_1'])
    ops.append(op)
    ###########################

    return [Layer(ops)]

In [10]:
# These operations are responsible for applying the rotation gates on the qubits, as well as the CNOTs.
def ansatz_operations(q_map, parameters):
    computing_host_ids = list(q_map.keys())
    
    layers = []
    
    ops = []
    j = 0
    for host_id in computing_host_ids:
        for i in range(len(q_map[host_id])):
            op = Operation(
                name=Constants.SINGLE,
                qids=[q_map[host_id][i]],
                gate=Operation.CUSTOM,
                gate_param=rotational_gate(parameters[j]),
                computing_host_ids=[host_id])
            j += 1
            
            ops.append(op)

    layers.append(Layer(ops))
    
    ops = []
    
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map['QPU_1'][0], q_map['QPU_1'][1]],
        gate=Operation.CNOT,
        computing_host_ids=['QPU_1'])

    ops.append(op)

    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map['QPU_1'][0], q_map['QPU_0'][0]],
        gate=Operation.CNOT,
        computing_host_ids=['QPU_1', 'QPU_0'])

    ops.append(op)

    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map['QPU_1'][1], q_map['QPU_0'][1]],
        gate=Operation.CNOT,
        computing_host_ids=['QPU_1', 'QPU_0'])

    ops.append(op)

    layers.append(Layer(ops))
        
    return layers

### The Protocols

#### The Controller Protocols 

This function builds the complete circuit needed to carry out VQE

In [11]:
def prepare_qubits_and_apply_ansatz(q_map, parameters): 
    circuit = Circuit(q_map, initialisation_operations(q_map) + ansatz_operations(q_map, parameters))
    return circuit

This function details the first communication protocol that will be carried out by the ControllerHost
As shown from the function names, the host first schedules the observables internally, then sends out the circuit to all the ComputingHosts. Note that we didn't send the schedules as of just yet.

In [12]:
def controller_host_protocol_preparation_ansatz(host, q_map, params, terms):
    """
    Protocol for the controller host
    """
    host.schedule_expectation_terms(terms, q_map)
    
    host.generate_and_send_schedules(prepare_qubits_and_apply_ansatz(q_map, params))

#### The Computing Protocols

This is the simple communication protocol of the ComputingHosts where they simply receive a schedule of operations and later listen to the synchronization until it is time to run a specific operation (which can a gate or some communication with another party)

In [13]:
def computing_host_protocol(host):
    host.receive_schedule()

## Step 5: Run the circuit and get the Expectation Value

Let's now run all the different pieces from above and see if we get the expected expectation value, which is the value from the PennyLane tutorial.

In [14]:
def init_network():
    # Retrieve the QuNetSim network
    network = Network.get_instance()
    network.delay = 0
    network.start()

    # Initialize the synchronization clock
    clock = Clock.get_instance()

    # Initialize the statevector backend
    eqsn = EQSNBackend()
    
    # Initialize the controller host with the given ID
    controller_host = ControllerHost(
        host_id="host_1",
        backend=eqsn
    )

    # Create a network with the given number of QPUs, each with the given number of qubits
    computing_hosts, q_map = controller_host.create_distributed_network(
        num_computing_hosts=2, # Since we don't need the third QPU, we will just drop it from our simulation
        num_qubits_per_host=2)
    controller_host.start()

    # Add all the nodes to the QuNetSim network
    network.add_hosts([controller_host])
    network.add_hosts(computing_hosts)
    
    return clock, controller_host, computing_hosts, q_map

In [15]:
np.random.seed(0)
params = np.random.normal(0, np.pi, (4, 3))

In [16]:
params

array([[ 5.54193389,  1.25713095,  3.07479606],
       [ 7.03997361,  5.86710646, -3.07020901],
       [ 2.98479079, -0.47550269, -0.32427159],
       [ 1.28993324,  0.45252622,  4.56873497]])

### Running the circuit

In [17]:
# Make sure that the QuNetSim network has no nodes that can interfere with our operation
list_hosts = list(Network.get_instance().ARP.keys())

for key in list_hosts:
    Network.get_instance().remove_host(Network.get_instance().get_host(key))

In [18]:
# Initialize the network and the hosts
clock, controller_host, computing_hosts, q_map = init_network()

2021-07-18 00:58:21,356: Host QPU_0 started processing
2021-07-18 00:58:21,356: Host QPU_1 started processing
2021-07-18 00:58:21,357: Host host_1 started processing


In [19]:
# Run the first ControllerHost communication protocol on a thread
t1 = controller_host.run_protocol(
    controller_host_protocol_preparation_ansatz,
    (q_map, params, terms))

# Run the first ComputingHost communication protocol on a separate thread for each QPU
threads = []
for host in computing_hosts:
    threads.append(host.run_protocol(computing_host_protocol))

# Wait for all threads to finish
t1.join()
for thread in threads:
    thread.join()

2021-07-18 00:58:22,503: host_1 sends BROADCAST message
2021-07-18 00:58:22,514: sending ACK:1 from QPU_1 to host_1
2021-07-18 00:58:22,531: QPU_1 received {"QPU_0": [{"name": "PREPARE_QUBITS", "qids": ["q_0_0", "q_0_1"], "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "TWO_QUBIT", "qids": ["q_0_0", "q_0_1"], "cids": null, "gate": "cnot", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_1"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SEND_ENT", "qids": ["13373bb7-6cdf-42c2-bec0-a084c6182c78"], "cids": null, "gate": null, "gate_param": null, "computing_host_i

2021-07-18 00:58:22,536: sending ACK:1 from QPU_0 to host_1
2021-07-18 00:58:22,553: QPU_1 sends CLASSICAL to host_1 with sequence 0
2021-07-18 00:58:22,584: host_1 received ACK from QPU_1 with sequence number 0
2021-07-18 00:58:22,647: QPU_0 received {"QPU_0": [{"name": "PREPARE_QUBITS", "qids": ["q_0_0", "q_0_1"], "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "TWO_QUBIT", "qids": ["q_0_0", "q_0_1"], "cids": null, "gate": "cnot", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_1"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SEND_ENT", "qids": ["13373bb7-

2021-07-18 00:58:22,647: QPU_0 sends CLASSICAL to host_1 with sequence 0
2021-07-18 00:58:22,648: host_1 received ACK from QPU_0 with sequence number 0
2021-07-18 00:58:22,649: QPU_1 awaits classical ACK from host_1 with sequence 0
2021-07-18 00:58:22,650: sending ACK:1 from host_1 to QPU_1
2021-07-18 00:58:22,652: QPU_0 awaits classical ACK from host_1 with sequence 0
2021-07-18 00:58:22,655: host_1 received ACK with sequence number 0
2021-07-18 00:58:22,656: QPU_1 received ACK from host_1 with sequence number 0
2021-07-18 00:58:22,661: sending ACK:1 from host_1 to QPU_0
2021-07-18 00:58:22,661: host_1 received ACK with sequence number 0
2021-07-18 00:58:22,670: QPU_0 received ACK from host_1 with sequence number 0
2021-07-18 00:58:23,783: QPU_0 sends EPR to QPU_1
2021-07-18 00:58:23,816: QPU_0 awaits EPR ACK from QPU_1 with sequence 0
2021-07-18 00:58:24,593: sending ACK:1 from QPU_1 to QPU_0
2021-07-18 00:58:24,635: QPU_0 received ACK from QPU_1 with sequence number 0
2021-07-18 00:

In [20]:
# Let's see how many ticks we needed for the applying the ansatz
clock.ticks

22

In the horizontal speedup notebook, we only needed 4 ticks to run the circuit and the ansatz on one QPU. However, in this simulation, where a lot of communication took place around two different QPUs, it jumped up to 22! This shows how important it is to divide the needed qubits among the QPUs as efficiently as possible.

Let's now retrieve the statevector, and compute the expectation value.

In [21]:
indices = []

# Get the IDs of all the qubits of first QPU. (It doesn't make a difference which QPU we chose)
for qubit_id in computing_hosts[0].qubit_ids:
    indices.append(computing_hosts[0].get_qubit_by_id(qubit_id))

# We then call the statevector function from the backend
statevector = computing_hosts[0].backend.statevector(indices[0])[1]

statevector

array([-0.05088414+1.03151285e-01j, -0.11620332-4.72234584e-02j,
       -0.00880123+6.74185416e-04j, -0.0250929 +1.79903542e-01j,
        0.27528829-4.70214876e-01j,  0.0251783 +8.19364645e-03j,
        0.04181508-2.11935461e-04j,  0.00257486-3.82575746e-02j,
       -0.06037203+1.46257672e-01j, -0.16383676-5.41489273e-02j,
       -0.00641623+6.19532459e-05j, -0.02692871+1.29266326e-01j,
        0.33468472-6.70694147e-01j,  0.03531208+8.93468440e-03j,
        0.03033871+1.87513730e-03j,  0.00372379-2.76232419e-02j])

In [22]:
from interlinq.utils.vqe_subroutines import expectation_value

In [23]:
expectation_value(terms, statevector, 4)

(-0.8074697622991748+9.315495045212815e-19j)

That's not very far from PennyLane's simulation value of -0.88179557 Ha! Such a small difference should be reconciled during the optimization process.

## Optimise

Now we can go on to attempt and optimize the parameters until we minimize the expecation value.

In [24]:
def cost_fn(params):
    params = params.reshape(4, 3)
    
    network = Network.get_instance()
    network.delay = 0
    network.start()

    Clock.reset_clock()

    eqsn = EQSNBackend()

    controller_host = ControllerHost(
        host_id="host_1",
        backend=eqsn
    )

    computing_hosts, q_map = controller_host.create_distributed_network(
        num_computing_hosts=2,
        num_qubits_per_host=2)
    
    controller_host.start()

    network.add_hosts(computing_hosts)
    network.add_hosts([controller_host])
    
    #############################################################
    
    t1 = controller_host.run_protocol(controller_host_protocol_preparation_ansatz, (q_map, params, terms))

    threads = []

    for host in computing_hosts:
        threads.append(host.run_protocol(computing_host_protocol))

    t1.join()
    for thread in threads:
        thread.join()
        
    #############################################################
    
    for host in computing_hosts:
        network.remove_host(host)
    network.remove_host(controller_host)
    
    indices = []

    for qubit_id in computing_hosts[0].qubit_ids:
        indices.append(computing_hosts[0].get_qubit_by_id(qubit_id))

    statevector = computing_hosts[0].backend.statevector(indices[0])[1]
    
    total_exp = expectation_value(terms, statevector, 4)
    
    return np.real(total_exp)

In [25]:
np.random.seed(0)
params = np.random.normal(0, np.pi, (4, 3))

params

array([[ 5.54193389,  1.25713095,  3.07479606],
       [ 7.03997361,  5.86710646, -3.07020901],
       [ 2.98479079, -0.47550269, -0.32427159],
       [ 1.28993324,  0.45252622,  4.56873497]])

Let's see if the cost function returns only one final loss value as expected.

In [26]:
cost_fn(params)

2021-06-26 00:51:58,567: Host QPU_0 started processing
2021-06-26 00:51:58,567: Host QPU_1 started processing
2021-06-26 00:51:58,567: Host host_1 started processing
2021-06-26 00:51:58,570: host_1 sends BROADCAST message
2021-06-26 00:51:58,727: sending ACK:1 from QPU_0 to host_1
2021-06-26 00:51:58,742: sending ACK:1 from QPU_1 to host_1
2021-06-26 00:51:58,779: QPU_0 received {"QPU_0": [{"name": "PREPARE_QUBITS", "qids": ["q_0_0", "q_0_1"], "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "TWO_QUBIT", "qids": ["q_0_0", "q_0_1"], "cids": null, "gate": "cnot", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_1"], "cids": null, "gate": "X", "gate_param

2021-06-26 00:51:58,779: QPU_0 sends CLASSICAL to host_1 with sequence 0
2021-06-26 00:51:58,781: QPU_1 received {"QPU_0": [{"name": "PREPARE_QUBITS", "qids": ["q_0_0", "q_0_1"], "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "TWO_QUBIT", "qids": ["q_0_0", "q_0_1"], "cids": null, "gate": "cnot", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_1"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SEND_ENT", "qids": ["f4d4ba40-0f73-46bb-9d1b-ed4b239056ae"], "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0", "QPU_1"], "pre_allocated_qub

2021-06-26 00:51:58,781: QPU_1 sends CLASSICAL to host_1 with sequence 0
2021-06-26 00:51:58,782: host_1 received ACK from QPU_0 with sequence number 0
2021-06-26 00:51:58,783: host_1 received ACK from QPU_1 with sequence number 0
2021-06-26 00:51:58,783: QPU_0 awaits classical ACK from host_1 with sequence 0
2021-06-26 00:51:58,786: sending ACK:1 from host_1 to QPU_0
2021-06-26 00:51:58,787: QPU_1 awaits classical ACK from host_1 with sequence 0
2021-06-26 00:51:58,788: sending ACK:1 from host_1 to QPU_1
2021-06-26 00:51:58,790: host_1 received ACK with sequence number 0
2021-06-26 00:51:58,792: host_1 received ACK with sequence number 0
2021-06-26 00:51:58,836: QPU_0 received ACK from host_1 with sequence number 0
2021-06-26 00:51:58,842: QPU_1 received ACK from host_1 with sequence number 0
2021-06-26 00:51:59,750: QPU_0 sends EPR to QPU_1
2021-06-26 00:51:59,777: QPU_0 awaits EPR ACK from QPU_1 with sequence 0
2021-06-26 00:52:00,553: sending ACK:1 from QPU_1 to QPU_0
2021-06-26 00

-0.807469762299175

Looks good! Let's disable the `Logger` to avoid clutter during the optimization process.

In [27]:
Logger.DISABLED = True

It should be noted that the networking nature of this simulation makes it tricky to use gradient-based approaches for optimization, as Interlin-q does not support it yet. So we will go for non-gradient based optimizers. 

### SciPy Optimisers

SciPy optimizers from the `minimize` API are too slow for the purely network version, thus they are not included in this notebook.

### Scikit-Quant Optimisers

This library is specialized for non-gradient based optimization of parameterized quantum circuits, and thus is perfect for our needs. Here, the library takes as input `budget` value, which determines how "long" it should attempt minimizing the expectation value.

**Note: the optimization can take up to 15 mins on an average computer.**

In [28]:
from skquant.interop.scipy import *

In [29]:
# We need to bound our parameters for the optimization
bounds = np.array([-3*np.pi, 3*np.pi])
bounds = np.tile(bounds, (4*3, 1))

np.random.seed(0)
params = np.random.normal(0, np.pi, (4, 3))

# The library must receive the parameters as a flat 1D array
flattened_parameters = params.flatten()
flattened_parameters

array([ 5.54193389,  1.25713095,  3.07479606,  7.03997361,  5.86710646,
       -3.07020901,  2.98479079, -0.47550269, -0.32427159,  1.28993324,
        0.45252622,  4.56873497])

Let's try the first optimizer.

In [30]:
budget = 100
minimum_bobyqa = minimize(cost_fn, flattened_parameters, method=pybobyqa, bounds=bounds, options={'budget' : budget})

minimum_bobyqa

     fun: -1.1329627749839424
 message: 'completed'
    nfev: 100
  status: 0
 success: True
       x: array([ 6.0388495 , -0.09447084,  3.57171167,  6.54305801,  6.27915045,
       -3.56712461,  3.4817064 , -0.2274104 ,  0.28697217,  1.78684884,
       -0.01450685,  4.07181936])

That's very close to PennyLane's -1.13613394 Ha!

Let's try the two other optimizers.

In [31]:
budget = 100
minimum_imfil = minimize(cost_fn, flattened_parameters, method=imfil, bounds=bounds, options={'budget' : budget})

minimum_imfil

     fun: -1.016230585889092
 message: 'completed'
    nfev: 105
  status: 0
 success: True
       x: array([ 5.54193389,  5.96951993,  5.43099055,  7.03997361,  5.86710646,
        6.35456895,  2.98479079, -0.47550269, -0.32427159,  1.28993324,
        0.45252622,  4.56873497])

In [32]:
budget = 100
minimum_snobfit = minimize(cost_fn, flattened_parameters, method=snobfit, bounds=bounds, options={'budget' : budget})

minimum_snobfit

     fun: -0.9093011711618758
 message: 'completed'
    nfev: 121
  status: 0
 success: True
       x: array([-7.67176926e-01, -6.06635258e+00,  5.90556587e-01, -4.01872532e-01,
        1.31946891e-03, -3.19481123e+00,  3.95916073e+00, -3.13279619e-01,
        2.32226529e-01, -3.31563689e+00, -5.25544469e+00,  2.24743255e+00])

It seems that a budget of 100 was not sufficient for them, and thus were not able to reach the same minimum value as that of BOBYQA.